In [ ]:
import os

# Define the path to the folder containing your Verilog files
folder_path = 'encoded'

# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


In [ ]:
# import os
# import json
# import re

# def label_verilog_file(file_name):
#     label_mapping = {
#         'adder': 0, 'ALU': 1, 'and': 2, 'bcd': 3, 'comparator': 4, 'decoder': 5,
#         'encoder': 6, 'mult': 7, 'mux': 8, 'nand': 9, 'nor': 10, 'not': 11, 'or': 12,
#         'pe': 13, 'seg': 14, 'sub': 15, 'xnor': 16, 'xor': 17
#     }
#     pattern = r"([a-zA-Z]+)(\d+)?"
#     match = re.match(pattern, file_name)
#     if match:
#         base_name = match.group(1)
#         if base_name in label_mapping:
#             return label_mapping[base_name]
#     return None

# def add_label_to_verilog_file(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, "r") as file:
#             loaded_data = json.load(file)
#             label = label_verilog_file(os.path.basename(file_path))
#             if label is not None and [label] not in loaded_data:  # Check if label already exists
#                 loaded_data.append([label])  # Add label as the third list
#                 with open(file_path, "w") as file:
#                     json.dump(loaded_data, file)
#                 return True
#     return False

# for file_path in verilog_files:
#     if not add_label_to_verilog_file(file_path):
#         print(f"Failed to label: {file_path}")
